<a href="https://colab.research.google.com/github/raymondwcs/embedding/blob/master/hong_kong_fastText_using_spaCy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U --quiet spacy
!python -m spacy download zh_core_web_lg

     |████████████████████████████████| 5.8 MB 4.2 MB/s 
     |████████████████████████████████| 10.1 MB 69.1 MB/s 
     |████████████████████████████████| 456 kB 58.0 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 621 kB 59.6 MB/s 
     |████████████████████████████████| 603.8 MB 9.0 kB/s 
     |████████████████████████████████| 2.4 MB 4.2 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_lg')


In [13]:
text01 = '特首'
text02 = '劉德華'
text03 = '男人'
text04 = '公仔麵'
text05 = '股票'

In [14]:
import spacy
import torch

nlp = spacy.load('zh_core_web_lg')
doc1 = nlp(text01)
doc2 = nlp(text02)
doc3 = nlp(text03)

print(doc1.similarity(doc2))
print(doc1.similarity(doc3))

print(torch.dist(torch.tensor(doc1.vector),torch.tensor(doc2.vector)))
print(torch.dist(torch.tensor(doc1.vector),torch.tensor(doc3.vector)))

0.226672768753576
-0.020377695782591902
tensor(44.6923)
tensor(71.7714)


# hong-kong-fastText
https://github.com/toastynews/hong-kong-fastText

In [4]:
!gdown --id 1NOkKfGMQPg7QfqNNj2Y46Xr4YbCbQZcZ

Downloading...
From: https://drive.google.com/uc?id=1NOkKfGMQPg7QfqNNj2Y46Xr4YbCbQZcZ
To: /content/toastynews.vec.gz
215MB [00:03, 56.2MB/s]


# Adding word vectors

https://spacy.io/usage/linguistic-features#adding-vectors

In [5]:
!mkdir -p ./tmp/hong-kong-fastText
!python -m spacy init vectors zh ./toastynews.vec.gz ./tmp/hong-kong-fastText

ℹ Creating blank nlp object for language 'zh'
[2021-09-03 01:33:41,577] [INFO] Reading vectors from toastynews.vec.gz
222906it [00:22, 9898.44it/s]
[2021-09-03 01:34:04,235] [INFO] Loaded vectors from toastynews.vec.gz
✔ Successfully converted 222906 vectors
✔ Saved nlp object with vectors to output directory. You can now use
the path to it in your config as the 'vectors' setting in [initialize].
/content/tmp/hong-kong-fastText


In [16]:
import torch
from spacy.vocab import Vocab
vocab = Vocab().from_disk("./tmp/hong-kong-fastText/vocab")

# vector01 = []
# vector02 = []
# vector03 = []

def get_vector(word):
  return torch.tensor(vocab.get_vector(word))

def closest_words(word, n=10):
  distances = []
  words = set(vocab.strings)
  for w in words:
    distances.append((w,torch.dist(get_vector(word),get_vector(w))))
  return sorted(distances, key=lambda w: w[1])[:n]

print(closest_words(text01))
print(closest_words(text02))
print(closest_words(text03))
print(closest_words(text04))
print(closest_words(text05))


tensor(26.0766)
tensor(35.2901)
[('特首', tensor(0.)), ('行政長官', tensor(14.7677)), ('梁振英', tensor(17.4522)), ('林鄭月娥', tensor(18.1291)), ('政務司司長', tensor(18.3308)), ('梁特首', tensor(18.7614)), ('曾俊華', tensor(18.9440)), ('雙特首', tensor(19.2513)), ('選特首', tensor(19.2669)), ('政司', tensor(19.2806))]
[('劉德華', tensor(0.)), ('周潤發', tensor(11.1872)), ('梁朝偉', tensor(11.2984)), ('王祖賢', tensor(11.6620)), ('林鳳嬌', tensor(11.7529)), ('郭富城', tensor(11.9565)), ('劉亞仁', tensor(12.2262)), ('吳倩蓮', tensor(12.3454)), ('林熙蕾', tensor(12.4014)), ('吳秀華', tensor(12.4324))]
[('男人', tensor(0.)), ('女人', tensor(8.6282)), ('臭男人', tensor(16.0667)), ('男人婆', tensor(16.4574)), ('女孩子', tensor(16.4829)), ('男客人', tensor(16.6189)), ('愛女人', tensor(16.8648)), ('男孩子', tensor(16.9009)), ('像男人', tensor(16.9187)), ('女犯人', tensor(17.3224))]
[('公仔麵', tensor(0.)), ('出前一丁', tensor(8.7252)), ('豬脾飯', tensor(11.1052)), ('撈檬粉', tensor(11.1255)), ('咖喱魚蛋', tensor(11.3871)), ('火腿蛋', tensor(11.3936)), ('湯意粉', tensor(11.5292)), ('魚旦粉', tensor(11.5393

In [17]:
fastText = spacy.load('./tmp/hong-kong-fastText')
doc4 = fastText(text04)
doc5 = fastText('出前一丁')

print(doc4.similarity(doc5))
print(torch.dist(torch.tensor(doc4.vector),torch.tensor(doc5.vector)))

0.1479973929449428
tensor(24.8274)
